In [12]:
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager
import time  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import pandas as pd
import requests
from selenium.webdriver.support.events import EventFiringWebDriver, AbstractEventListener

class MyListener(AbstractEventListener):
    def before_navigate_to(self, url, driver):
        print("Before navigate to %s" % url)
    def after_navigate_to(self, url, driver):
        print("After navigate to %s" % url)

product_list=[]   

#Function to iterate within a category/section
def iterate_category(): 
    global product_list
    time.sleep(3)
    cat_name=driver.find_element(by=By.CSS_SELECTOR,value="h1.category-navs__current").text
    #cat_name=category.find_element(by=By.CSS_SELECTOR,value=".name").text
    subcategories=driver.find_elements(by=By.CSS_SELECTOR,value=".category-list__item.no-child")
    length_subcat=len(subcategories)
    j=0
    while(True):
        driver.execute_script("window.scrollBy(0,-1500)", "")
        time.sleep(3)
        if(length_subcat==0):
            subcat_name=cat_name
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)", "")
            time.sleep(4)
            driver.execute_script("window.scrollTo(0,-document.body.scrollHeight)", "")
#             htmlelement= driver.find_element_by_tag_name('html')
#             htmlelement.send_keys(Keys.END)
#             time.sleep(3)
#             htmlelement.send_keys(Keys.HOME)
        else:    
            subcategories=driver.find_elements(by=By.CSS_SELECTOR,value="li.category-list__item.no-child")
            subcat_name=subcategories[j].find_element(by=By.CSS_SELECTOR,value=".name").text
            subcategories[j].click()
#             htmlelement= driver.find_element_by_tag_name('html')
#             htmlelement.send_keys(Keys.END)
#             time.sleep(3)
#             htmlelement.send_keys(Keys.HOME)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)", "")
            time.sleep(4)
            driver.execute_script("window.scrollTo(0,-document.body.scrollHeight)", "")
        time.sleep(3)
        products=driver.find_elements(by=By.CLASS_NAME,value="product__wrapper")
        length_prod=len(products)
        print(length_prod)
        time.sleep(3)
        for k in range(length_prod):#Iterating through the products
            products=driver.find_elements(by=By.CLASS_NAME,value="product__wrapper")
            time.sleep(3)
            products[k].click()
            time.sleep(3)
            units=driver.find_elements(by=By.CSS_SELECTOR,value=".css-901oao.r-1niwhzg")
            for l in range(len(units)):
                time.sleep(3)
                units[l].click()
                time.sleep(3)
                page_source = driver.page_source
                soup = BeautifulSoup(page_source, 'html.parser')
                div_ele=soup.find('div',attrs={"class" : "css-1dbjc4n r-ymttw5"})
                mrp_ele=div_ele.find('div',attrs={"class" : "r-142tt33"})
                if mrp_ele:
                    mrp=mrp_ele.text
                else:
                    mrp=div_ele.find('div',attrs={"class" : "r-1d4mawv"}).text
                details={
                  'Category':cat_name,
                  'Subcategory':subcat_name,
                  'Image File Names':[],    
                  'Name':div_ele.find('div',attrs={"class" : "r-1vzi8xi"}).text,  
                  'Quantity':div_ele.find('div',attrs={"class" : "r-5oul0u"}).text,
                  'MRP':mrp, 
                  'Selling Price':div_ele.find('div',attrs={"class" : "r-1d4mawv"}).text
                }
                img_text_list=[]
                images=driver.find_elements(by=By.XPATH,value="//div[(starts-with(@class, 'slick-slide')) and not(contains(@class,'slick-cloned'))]  ")
                time.sleep(3)
                for i in range(len(images)):
                    time.sleep(2)
                    img=images[i].find_element(by=By.CSS_SELECTOR,value=".img-loader__img.img-loader__img--no-transition.img-loader__img--shown")
                    unit_text=units[l].text
                    img_text=details['Name']+unit_text+str(i+1)+'.jpg'
                    # initializing bad_chars_list
                    bad_chars=['#','<','>','[',']','|',':','{','}','/','~']
                    for i in bad_chars :
                        img_text = img_text.replace(i, '')
                    img_text_list.append(img_text)
                    src=img.get_attribute('src')
                    print(src)
                    headers = headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Cafari/537.36'}
                    pic = requests.get(src, headers=headers)
                    with open(img_text, 'wb') as photo:
                        photo.write(pic.content)
                details['Image File Names']=img_text_list                
                product_list.append(details)
            time.sleep(3)
            #break
            driver.back()
            time.sleep(3)
            break
        j=j+1
        if(j>=length_subcat):
            break
        break
    for j in range(length_subcat):
        break
        driver.back()
        time.sleep(3)
    if(length_subcat==0):
        driver.back()
        time.sleep(3)
        
print("sample test case started")  
driver = webdriver.Chrome(ChromeDriverManager().install())
#driver=webdriver.firefox()  
#driver=webdriver.ie()  

#maximize the window size  
driver.maximize_window()  

#navigate to the url  
driver.get("https://blinkit.com/")  
time.sleep(5)

#Clicking on 'Detect my Location' button.
#driver.find_element(by=By.CSS_SELECTOR,value=".btn.location-box.mask-button").click()

#Entering location and selecting it.
driver.find_element(by=By.CSS_SELECTOR,value=".location-search-input-google-v1.pac-target-input").send_keys("Gurugram Haryana,India")
time.sleep(5)
driver.find_element(by=By.CSS_SELECTOR,value=".pac-item").click()
time.sleep(5)

#Scrolling the home page
event = EventFiringWebDriver(driver,MyListener())
event.execute_script("document.querySelector('div.css-1dbjc4n.r-150rngu').scrollTop=800")
time.sleep(5)

element=driver.find_element(by=By.CSS_SELECTOR,value=".css-1dbjc4n.r-zs9c5m")
time.sleep(3)
sections=element.find_elements(by=By.CLASS_NAME,value="css-9pa8cd")
length_sec=len(sections)
time.sleep(3)

for i in range(length_sec):#Iterating through the categories
    #Scrolling the home page
    event = EventFiringWebDriver(driver,MyListener())
    event.execute_script("document.querySelector('div.css-1dbjc4n.r-150rngu').scrollTop=800")
    time.sleep(5)
    element=driver.find_element(by=By.CSS_SELECTOR,value=".css-1dbjc4n.r-zs9c5m")
    time.sleep(3)
    sections=element.find_elements(by=By.CLASS_NAME,value="css-9pa8cd")
    time.sleep(3)
    sections[i].click()
    time.sleep(3)
    iterate_category()
    #break 
    if(i==(length_sec-4)):
        driver.back()
    driver.back()
    time.sleep(3) 
driver.close()    
df=pd.DataFrame(product_list)
df.to_csv('product_list.csv')



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome


sample test case started


Driver [/home/dell/.wdm/drivers/chromedriver/linux64/98.0.4758.102/chromedriver] found in cache
/tmp/ipykernel_4304/23750841.py:125: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


91


WebDriverException: Message: chrome not reachable
  (Session info: chrome=98.0.4758.102)
Stacktrace:
#0 0x55afc3257b33 <unknown>
#1 0x55afc2d2056f <unknown>
#2 0x55afc2d0f184 <unknown>
#3 0x55afc2d0fb89 <unknown>
#4 0x55afc2d118e2 <unknown>
#5 0x55afc2d0a076 <unknown>
#6 0x55afc2d21ad2 <unknown>
#7 0x55afc2d869b1 <unknown>
#8 0x55afc2d73f53 <unknown>
#9 0x55afc2d49a0a <unknown>
#10 0x55afc2d4aad5 <unknown>
#11 0x55afc32892fd <unknown>
#12 0x55afc32a24bb <unknown>
#13 0x55afc328b0d5 <unknown>
#14 0x55afc32a3145 <unknown>
#15 0x55afc327eaaf <unknown>
#16 0x55afc32bfba8 <unknown>
#17 0x55afc32bfd28 <unknown>
#18 0x55afc32da48d <unknown>
#19 0x7fbb63536609 <unknown>
